# Machine Translation

In diesem Notebook möchten wir uns mit der Königsdisziplin des Natural Language Processings beschäftigen, der maschinellen Übersetzung.
Vermutlich gibt es keine NLP-Anwendung, die einerseits vielen bekannt ist, anderseits durch Deep Learning und neuronalen Netzen einen solchen Aufschwung bekommen hat.
Vor dem Siegeszug der neuronalen Netzen, wurde maschinelle Übersetzung deshalb als schwierig angesehen, weil sie alle Teilaspekte von Sprache beinhaltet. Neben grammatikalischer Korrektheit, sollen maschinell übersetzte Texte den Sinn des Originaltextes wiedergeben und darüberhinaus auch den Subtext, wie Ironie, Witz, erfassen.
Ob letzteres einfach gelingt, sei hier mal dahingestellt.

## Daten
Im Vergleich zu den bisherigen Notebooks sind die Daten für machine translation etwas dröge. Daher möchten wir uns hier nicht allzu lange aufhalten. Speichert zunächst folgende Datei ab und entpackt sie: http://www.manythings.org/anki/fra-eng.zip

Danach wollen wir die Daten einlesen. Das Format ist einfach, pro Zeile steht ein Satzpaar getrennt von einem Tab.

In [1]:
with open('fra.txt', 'r') as f:
    lines = f.readlines()

In [2]:
pairs = [tuple(l.split("\t")) for l in lines]

In [3]:
pairs_cleaned = [(e, f.strip()) for e, f in pairs]

Damit unser Modell weiß wo Anfang und Ende der französischen Sätze sind, müssen wir noch ein spezielles Start- (🏳️) und Endsymbol (🏴) einfügen:

In [4]:
pairs_cleaned_with_marker = [(e, "🏳️" + f + "🏴") for e, f in pairs_cleaned]

Speichert euch nun alle Character, die in den englischen Sätzen und in den französischen Sätzen in jeweils einem Set ab.

In [5]:
print(pairs_cleaned_with_marker[0])

('Go.', '🏳️Va !🏴')


In [6]:
english_characters = set()
french_characters = set()

for str_en, str_fr in pairs_cleaned_with_marker:
    [english_characters.add(char) for char in str_en]
    [french_characters.add(char) for char in str_fr]

english_characters = list(english_characters)
french_characters = list(french_characters)

print(english_characters)
print(french_characters)

['B', 'q', 'U', '0', '3', 'e', 'ö', 'I', 'p', 'n', '’', 'o', 'H', 'a', 'k', '₂', 'P', 'E', '?', 's', '1', 'G', '"', '!', ' ', '$', ',', 'S', '+', 'j', 'Y', '\xa0', '.', 'é', 'r', '—', 'M', '2', '4', 'A', 'O', ':', '\xad', 'b', '/', 'u', 'X', 't', '‘', 'm', 'Q', "'", 'C', 'º', 'Z', 'N', '5', '–', ';', 'c', 'D', 'F', 'd', 'J', 'w', '7', 'v', 'ú', 'V', 'W', '8', 'i', '&', '€', '9', 'K', 'l', 'x', 'а', '-', 'L', 'h', '%', 'g', 'T', 'z', '6', 'ç', 'y', 'R', 'f']
['B', 'ù', 'q', 'U', '0', '3', 'e', 'ö', 'I', '\u200b', 'p', 'n', '’', 'o', 'H', 'Ô', 'a', 'k', '₂', 'P', 'E', '?', 's', '1', 'G', '"', '!', ' ', '$', '‽', ',', 'S', '+', 'j', 'Y', 'À', '\xa0', 'è', 'Ç', '.', '…', 'é', 'r', 'M', '2', '4', 'A', '\u2009', 'O', 'î', ':', 'b', 'ô', '/', 'u', 'X', '️', 't', '‘', 'à', 'ê', 'm', 'Q', "'", 'û', '»', 'C', 'ï', 'Z', 'N', '5', '–', ';', 'c', 'D', 'F', 'd', 'ë', 'J', 'w', '7', 'v', '🏴', '(', 'V', 'W', '8', 'i', '&', ')', '9', 'С', '🏳', 'Ê', 'â', 'K', 'l', 'x', '\u202f', '-', 'L', 'h', 'g', '%',

Um unser Modell später mit passenden Eingabedaten füttern zu können, müssen wir nun noch die jeweiligen Vektorlängen für den encoder und den decoder bestimmen. Speichert die Längen in den Variablen  `encoder_vector_len` und `decoder_vector_len` ab. Hinweis: uns Modell bekommt Vektoren, die die Characters pro Wort abbilden als Eingabe.

In [7]:
encoder_vector_len = len(english_characters)#TODO
decoder_vector_len = len(french_characters)#TODO
print(encoder_vector_len)
print(decoder_vector_len)

91
116


Um die Eingabevektoren bestimmen zu können, müssen wir uns noch Dictionaries bauen, die jeweils die Character auf die Komponenten des Vektors abbilden

In [8]:

english_token_index = {char:key for key, char in enumerate(english_characters)}
french_token_index = {char:key for key, char in enumerate(french_characters)}

print(english_token_index) #token2idx
print(french_token_index)

{'B': 0, 't': 47, 'q': 1, 'U': 2, '‘': 48, 'g': 83, '0': 3, '3': 4, 'e': 5, 'ö': 6, 'I': 7, 'm': 49, 'Q': 50, 'p': 8, '’': 10, 'o': 11, 'а': 78, 'H': 12, 'C': 52, 'a': 13, '₂': 15, 'º': 53, 'Z': 54, '5': 56, 'w': 64, '–': 57, ';': 58, 'P': 16, 'z': 85, ' ': 24, 'i': 71, '?': 18, 'c': 59, 'D': 60, 'F': 61, '1': 20, 'G': 21, '!': 23, 'J': 63, 'N': 55, '7': 65, '"': 22, 'k': 14, 'v': 66, '$': 25, ',': 26, 'S': 27, '+': 28, 'ú': 67, 'V': 68, 'j': 29, '8': 70, 'Y': 30, '&': 72, '€': 73, '\xa0': 31, '9': 74, '.': 32, 'K': 75, 'é': 33, 'n': 9, 'l': 76, 'x': 77, 'r': 34, '—': 35, '-': 79, 'M': 36, "'": 51, 'h': 81, '2': 37, '%': 82, 'L': 80, 'T': 84, '4': 38, 'A': 39, 'd': 62, 'E': 17, 'O': 40, '6': 86, 's': 19, 'f': 90, 'ç': 87, 'W': 69, ':': 41, '\xad': 42, 'b': 43, 'y': 88, 'R': 89, '/': 44, 'u': 45, 'X': 46}
{'B': 0, 'ù': 1, 'q': 2, 'U': 3, '‘': 58, '0': 4, 'à': 59, 'ê': 60, '3': 5, 'e': 6, '%': 103, 'ö': 7, '\u200b': 9, 'm': 61, 'Q': 62, 'p': 10, "'": 63, 'o': 13, 'û': 64, 'H': 14, 'k': 1

Wir haben nun alle Dimensionen bestimmt, um die Trainingsdaten zu definieren. Dafür bauen wir uns drei Matrizen (eine als Eingabe für den Encoder, eine als Eingabe für den Decoder und eine als Ausgabe des Decoders) zusammen. Die Dimensionen dafür sind jeweils: Anzahl Sätze, Anzahl Wörter des längsten Satzes, Anzahl Characters.

In [9]:
import numpy as np

max_encoder_sentence_len = max((len(e) for e, _ in pairs_cleaned_with_marker))
max_decoder_sentence_len = max((len(f) for _, f in pairs_cleaned_with_marker))


encoder_input_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_encoder_sentence_len, encoder_vector_len),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_decoder_sentence_len, decoder_vector_len),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_decoder_sentence_len, decoder_vector_len),
    dtype='float32')

Befüllt nun die Matrizen. Die `encoder_input_data` und die `decoder_input_data`-Matrix soll für jeden Satz aus den Trainingsdaten, einen Eintrag mit einem Token-Vektor pro Wort enthalten.
Die `decoder_target_data`-Matrix soll den gleichen Inhalt wie die `decoder_input_data`-Matrix haben, allerdings um eine Position verschoben. (Schaut euch nochmal die Folien an, wir brauchen Platz für das Startsymbol) 

Die `encoder_input_data`-Matrix wird mit den englischen Daten befüllt, die beiden anderen mit den französischen Daten.



In [10]:
for i, (e, f) in enumerate(pairs_cleaned_with_marker):
    for t, char in enumerate(e):
        encoder_input_data[i, t, english_token_index[char]] = 1.
    for t, char in enumerate(f):
        decoder_input_data[i, t, french_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t-1, french_token_index[char]] = 1.

## Sequence-to-sequence Modelle

Im Gegensatz zu den bisherigen Notebooks, wollen wir hier mit dem Modell beginnen. Einerseits, weil die Daten eher dröge  (trocken) sind (englisch-fränzosische Satzpaare), andererseits, weil das Modell selbst komplizierter ist und wir hier die Functional-API von Keras benutzen müssen.

Im Gegensatz zu der Sequence-API, definiert man hier ein Modell in dem, den jeweiligen Layer mit dem Vorgänger Layer aufruft:
```

a = Input(...)
b = Dense(...)

t = b(a)


```
Als Rückgabe bekommt man einen Tensor.


Ganz generell gesprochen, möchten wir hier ein Seq2Seq-Modell aufbauen, das auf LSTMs basiert. Wie auf den Folien beschrieben wurde, teilt sich so ein Modell in einen Encoder und Decoder auf. 
Wir beginnen die Implementierung mit dem Encoder.

### Encoder

Auch das komplexeste Modell fängt mit einer Eingabe ein.

Definiert einen Input-Layer, der als Eingabedimension, Sätze beliebiger Länge als Vektoren nimmt. Eine Komponente im Vektor repräsentiert einen Character. Unbekannte Teile der Eingabedimensionen kann man mit `None` definieren.

In [11]:
from keras.layers import Input

encoder_input = Input(shape=(None, encoder_vector_len))

Using TensorFlow backend.


Als nächstes definiert einen LSTM-Layer, der 256 hidden units hat. Die Ausgabe des Layers soll später an den Decoder 'verfüttert' werden. Seht in der Dokumentation nach wie man den Thought-Vektor zurückbekommen kann (https://keras.io/layers/recurrent/)

In [12]:
from keras.layers import LSTM
lstm_hu = 256
encoder = LSTM(units=lstm_hu, return_state=True)

Verbindet nun den `encoder_input`-Layer mit dem `encoder`-Layer

In [13]:
_, state_h, state_c = encoder(encoder_input)

Instructions for updating:
Colocations handled automatically by placer.


Der Encoder wäre damit fertig implementiert.

### Decoder

Auch der Decoder fängt mit einem Input-Layer an. Definiert einen Input-Layer analog zum Encoder.

In [14]:
decoder_input = Input(shape=(None, decoder_vector_len))

Wie auch im Encoder, ist das Herzstück ein LSTM-Layer. Definiert einen LSTM-Layer mit 256 hidden units. Seht in der Dokumentation nach, wie man neben den Thought-Vektoren, auch die komplette Ausgabe-Sequence zurückbekommt.

In [15]:
decoder_lstm = LSTM(256, return_state=True, return_sequences=True)

Verbindet nun den `decoder_input`-Layer mit dem `decoder_lstm`-Layer. An welcher der drei Ausgaben sind wir hier interessiert? Außerdem müssen wir hier, den Thought-Vektor des Encoders mit übergeben, dies geschieht über den `initial_state`-Parameter

In [16]:
sequences ,_,_ = decoder_lstm(decoder_input, initial_state=[state_h,state_c])

Den Abschluss unseres Modells bildet ein Dense-Layer, der gleich viele `hidden_units` wie der Input-Layer des Decoders hat. Als Aktivierungsfunktion nehmen wir die Softmax-Funktion.

In [17]:
from keras.layers import Dense

decoder_layer = Dense(decoder_vector_len, activation="softmax")

Verbindet diesen Layer mit den bisherigen Decoder sequences.

In [18]:
decoder = decoder_layer(sequences)

Wir haben nun alle Elemente für ein Sequence-To-Sequence-Modell zusammen. Im letzten Schritt bauen wir das ganze Modell zusammen.

In [19]:
TRAIN = True
if not TRAIN:
    import keras
    from keras.models import load_model
    from keras.utils import CustomObjectScope
    from keras.initializers import glorot_uniform

    with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model('model15.h5')

Alternativ statt laden trainieren:

In [20]:
from keras import Model
if TRAIN:
    model = Model(inputs=[encoder_input, decoder_input], outputs=[decoder])
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

Fertig! Nun kann das Modell trainiert werden.

In [21]:
if TRAIN:
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=256,
              epochs=150,
              validation_split=0.2)

    model.save('model150.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 133704 samples, validate on 33426 samples
Epoch 1/150
133704/133704 [==============================] - 412s 3ms/step - loss: 0.2106 - val_loss: 0.3250
Epoch 2/150
133704/133704 [==============================] - 390s 3ms/step - loss: 0.1486 - val_loss: 0.2728
Epoch 3/150
133704/133704 [==============================] - 378s 3ms/step - loss: 0.1249 - val_loss: 0.2458
Epoch 4/150
133704/133704 [==============================] - 389s 3ms/step - loss: 0.1118 - val_loss: 0.2279
Epoch 5/150
133704/133704 [==============================] - 379s 3ms/step - loss: 0.1039 - val_loss: 0.2193
Epoch 6/150
133704/133704 [==============================] - 389s 3ms/step - loss: 0.0983 - val_loss: 0.2081
Epoch 7/150
133704/133704 [==============================] - 379s 3ms/step - loss: 0.0949 - val_loss: 0.2035
Epoch 8/150
133704/133704 [==============================] -

133704/133704 [==============================] - 391s 3ms/step - loss: 0.0558 - val_loss: 0.1700
Epoch 71/150
133704/133704 [==============================] - 390s 3ms/step - loss: 0.0555 - val_loss: 0.1712
Epoch 72/150
133704/133704 [==============================] - 390s 3ms/step - loss: 0.0553 - val_loss: 0.1710
Epoch 73/150
133704/133704 [==============================] - 391s 3ms/step - loss: 0.0552 - val_loss: 0.1715
Epoch 74/150
133704/133704 [==============================] - 391s 3ms/step - loss: 0.0550 - val_loss: 0.1711
Epoch 75/150
133704/133704 [==============================] - 390s 3ms/step - loss: 0.0549 - val_loss: 0.1725
Epoch 76/150
133704/133704 [==============================] - 390s 3ms/step - loss: 0.0548 - val_loss: 0.1734
Epoch 77/150
133704/133704 [==============================] - 390s 3ms/step - loss: 0.0547 - val_loss: 0.1735
Epoch 78/150
133704/133704 [==============================] - 391s 3ms/step - loss: 0.0546 - val_loss: 0.1718
Epoch 79/150
133704/133

133704/133704 [==============================] - 391s 3ms/step - loss: 0.0516 - val_loss: 0.1802
Epoch 145/150
133704/133704 [==============================] - 391s 3ms/step - loss: 0.0505 - val_loss: 0.1815
Epoch 146/150
133704/133704 [==============================] - 391s 3ms/step - loss: 0.0503 - val_loss: 0.1799
Epoch 147/150
133704/133704 [==============================] - 391s 3ms/step - loss: 0.0502 - val_loss: 0.1826
Epoch 148/150
133704/133704 [==============================] - 391s 3ms/step - loss: 0.0502 - val_loss: 0.1810
Epoch 149/150
133704/133704 [==============================] - 392s 3ms/step - loss: 0.0501 - val_loss: 0.1814
Epoch 150/150
133704/133704 [==============================] - 392s 3ms/step - loss: 0.0500 - val_loss: 0.1828


/usr/local/lib/python3.5/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


### Inferenz

Um mit dem trainierten Modell Texte übersetzen zu können, müssen wir nun noch unser bisheriges Modell etwas umbauen.
Im ersten Schritt definieren wir uns ein `Model` das einen Eingabetext encodiert. Als Eingabe hat dieses `Model` den bisherigen `encoder_input`-Layer und die Thought-Vektoren als Output. 


In [22]:
encoder_model = Model(encoder_input, [state_h, state_c])

In [23]:
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 91)          0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 356352    
Total params: 356,352
Trainable params: 356,352
Non-trainable params: 0
_________________________________________________________________


Etwas komplizierter ist das Decoder-Modell. Als Input nimmt es zum einen die Thought-Vektoren des Encoders, zum anderen den `decoder_input`-Layer von oben.
Definiert zunächst zwei Input-Layer für die Thought-Vektoren.

In [24]:
decoder_state_input_h = Input(shape=(lstm_hu,))
decoder_state_input_c = Input(shape=(lstm_hu,))

Setzt diese beiden Input-Layer als `initial_state` und den `decoder_input`-Layer als Input in das `decoder_lstm` von oben ein.

In [25]:
decoder_sequences, state_h, state_c = decoder_lstm(decoder_input, initial_state=[decoder_state_input_h, decoder_state_input_c])

Setzt nun den `decoder_outputs`-Tensor in den `decoder_layer` von oben ein.

In [26]:
decoder_outputs = decoder_layer(decoder_sequences)

Nun können wir ein `Model` zusammenbauen, das uns als Decoder dient. Input ist hier der `decoder_input`-Layer, zusammen mit den `decoder_state_input_h` und dem `decoder_state_input_c`-Tensor. Als Output dient uns der `decoder_outputs`-Tensor und der `state_h` mit dem `state_c`-Tensor. 

In [27]:
decoder_model = Model(inputs=[decoder_input, decoder_state_input_h, decoder_state_input_c], outputs=[decoder_outputs, state_h, state_c])

Ein kleiner Zwischenschritt ist noch nötig, bevor wir mit dem Modell Texte übersetzen können. Damit wir aus unseren Vektoren wieder Texte bekommen, müssen wir noch das Dictionary `reverse_french_token_index` befüllen, das die "Umkehrung" von `french_token_index` ist.

In [28]:
reverse_french_token_index = {idx:token for token, idx in french_token_index.items()} #idx2token
reverse_english_token_index = {idx:token for token, idx in english_token_index.items()} #idx2token

In [29]:
print(reverse_french_token_index)

{0: 'B', 1: 'ù', 2: 'q', 3: 'U', 4: '0', 5: '3', 6: 'e', 7: 'ö', 8: 'I', 9: '\u200b', 10: 'p', 11: 'n', 12: '’', 13: 'o', 14: 'H', 15: 'Ô', 16: 'a', 17: 'k', 18: '₂', 19: 'P', 20: 'E', 21: '?', 22: 's', 23: '1', 24: 'G', 25: '"', 26: '!', 27: ' ', 28: '$', 29: '‽', 30: ',', 31: 'S', 32: '+', 33: 'j', 34: 'Y', 35: 'À', 36: '\xa0', 37: 'è', 38: 'Ç', 39: '.', 40: '…', 41: 'é', 42: 'r', 43: 'M', 44: '2', 45: '4', 46: 'A', 47: '\u2009', 48: 'O', 49: 'î', 50: ':', 51: 'b', 52: 'ô', 53: '/', 54: 'u', 55: 'X', 56: '️', 57: 't', 58: '‘', 59: 'à', 60: 'ê', 61: 'm', 62: 'Q', 63: "'", 64: 'û', 65: '»', 66: 'C', 67: 'ï', 68: 'Z', 69: 'N', 70: '5', 71: '–', 72: ';', 73: 'c', 74: 'D', 75: 'F', 76: 'd', 77: 'ë', 78: 'J', 79: 'w', 80: '7', 81: 'v', 82: '🏴', 83: '(', 84: 'V', 85: 'W', 86: '8', 87: 'i', 88: '&', 89: ')', 90: '9', 91: 'С', 92: '🏳', 93: 'Ê', 94: 'â', 95: 'K', 96: 'l', 97: 'x', 98: '\u202f', 99: '-', 100: 'L', 101: 'h', 102: 'g', 103: '%', 104: 'T', 105: '«', 106: 'É', 107: 'z', 108: '6', 1

Nun können wir eine Funktion `translate` implementieren, die als Eingabe einen englischen Text als Vektoren nimmt.

In [30]:

#first start: predict_char(french_token_index["🏳"], thought[0], thought[1]) -> french_char_idx, h, c
#every iter: predict_char(french_char_idx, h, c) -> french_char_idx, h, c


def predict_char(prev_char_idx, state_h, state_c):
    french_char_onehot = np.zeros((1, 1, decoder_vector_len))
    french_char_onehot[0, 0, prev_char_idx] = 1
    predicted_chars_probs, h, c = decoder_model.predict([french_char_onehot, state_h, state_c])
    predicted_french_char_idx = np.argmax(predicted_chars_probs)
    predicted_french_char = reverse_french_token_index[predicted_french_char_idx]
    return predicted_french_char_idx, predicted_french_char, h, c

def translate(english_text):
    # Wandelt mit dem encoder_model, `english_text` in einen Thought-Vektor um
    thought = encoder_model.predict(english_text)
    
    translation = ""
    
    predicted_french_char_idx, h, c = french_token_index["🏳"], thought[0], thought[1]
    
    while predicted_french_char_idx != french_token_index["🏴"]:
        predicted_french_char_idx, predicted_french_char, h, c = predict_char(predicted_french_char_idx, h, c)
        #print(h[0][0])
        translation += predicted_french_char
    
    return translation[:-1]

    """
    ----------
    # Als Input für den Decoder, wird zum einen der thought-Vektor benötigt, 
    # zum anderen die Vektor-Repräsentation des Startsymbols
    french_text = np.zeros((1, 1, decoder_vector_len)) #TODO
    
    # Belegt die Komponente des Start-Symbols mit 1
    french_text[0, 0, french_token_index["🏳"]] = 1.  #TODO
    
    # und übergebt beides an den Decoder
    chars, h, c = decoder_model.predict([french_text] + thought)
    print(chars)
    # chars beinhaltet eine 1 x 1 x decoder_vector_len Matrix, die für jeden französische Character
    # eine Art "Wahrscheinlichkeit" angibt.
    # Findet die Komponente, mit der größten Wahrscheinlichkeit und wandelt sie in ein Character um
    french_char_idx = np.argmax(chars) #TODO
    french_char = reverse_french_token_index[french_char_idx]
    print(french_char_idx, french_char)
    
    -------------------
    # die nächste Prediction wäre dann
    french_text = np.zeros((1, 1, decoder_vector_len))
    french_text[0, 0, french_char_idx] = 1.
    chars, h, c = decoder_model.predict([french_text, h, c])
    french_char_idx = np.argmax(chars) #TODO
    french_char = reverse_french_token_index[french_char_idx]
    print(french_char_idx, french_char)
    -----------------
    """
    # TODO: baut eine Schleife mit geeigneter Abbruchbedingung, die die Schritte von oben umfasst 
    # und dabei aus french_char einen übersetzen Text zusammenbaut

In [31]:
english_text = np.array([encoder_input_data[10]])
print(english_text.shape, len(english_text[0]), encoder_vector_len)

french_text = translate(english_text)

print("-- English --")
#print(pairs_cleaned_with_marker[10])
print("".join([reverse_english_token_index[np.argmax(char)] for char in english_text[0]]))
print("-- French --")
print(french_text)


(1, 286, 91) 286 91
-- English --
Stop!BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
-- French --
️Arrêtez !


In [32]:
print(pairs_cleaned_with_marker[0:3])

[('Go.', '🏳️Va !🏴'), ('Hi.', '🏳️Salut !🏴'), ('Run!', '🏳️Cours\u202f!🏴')]


In [43]:
output_string = ""

for text in encoder_input_data:
    english_text = np.array([text])
    english_text_string = "".join([reverse_english_token_index[np.argmax(char)] for char in english_text[0]])
    french_text = translate(english_text)
    output_string += english_text_string + "|||||" + french_text + "\n-----------------------\n"
    
with open("translation.txt", "w") as f:
    f.write(output_string)